In [1]:
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import gensim
from nltk.corpus import stopwords
import re


C:\Users\HP\AppData\Roaming\Python\Python38\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
train_df=pd.read_csv('../ZS_associate_NLP/dataset/train_file.csv')
test_df=pd.read_csv('../ZS_associate_NLP/dataset/test_file.csv')
display(train_df.head(5))
display(train_df.isna().sum())

train_df['Source'] = train_df.Source.fillna(train_df["Source"].mode().iloc[0])
train_df.isna().sum()



,IDLink,Title,Headline,Source,Topic,PublishDate,Facebook,GooglePlus,LinkedIn,SentimentTitle,SentimentHeadline
0,Tr3CMgRv1N,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,2002-04-02 00:00:00,-1,-1,-1,0.000000,-0.053300
1,Wc81vGp8qZ,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,2008-09-20 00:00:00,-1,-1,-1,0.208333,-0.156386
2,zNGH03CrZH,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,2012-01-28 00:00:00,-1,-1,-1,-0.425210,0.139754
3,3sM1H0W8ts,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,2015-03-01 00:06:00,-1,-1,-1,0.000000,0.026064
4,wUbnxgvqaZ,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,2015-03-01 00:11:00,-1,-1,-1,0.000000,0.141084


IDLink                 0
Title                  0
Headline               0
Source               175
Topic                  0
PublishDate            0
Facebook               0
GooglePlus             0
LinkedIn               0
SentimentTitle         0
SentimentHeadline      0
dtype: int64

IDLink               0
Title                0
Headline             0
Source               0
Topic                0
PublishDate          0
Facebook             0
GooglePlus           0
LinkedIn             0
SentimentTitle       0
SentimentHeadline    0
dtype: int64

# display(test_df.head())
display(test_df.isna().sum())


In [5]:
topic_list=train_df['Topic'].tolist()
topic_count=Counter(topic_list)
top_topic=topic_count.most_common()
x,y=map(list,zip(*top_topic))
plt.figure(figsize=(11,6))
# sns.barplot(x=y,y=x)


<Figure size 792x432 with 0 Axes>

<Figure size 792x432 with 0 Axes>

In [6]:
def preprocess_text(texts):
    texts = texts.lower() 
    texts = re.sub(r'[^\x00-\x7F]+',' ', texts) 
    splitwords = texts.split()
    splitwords = filter(lambda x: x[0]!= '@' , texts.split()) 
    splitwords = [word for word in splitwords if word not in set(stopwords.words('english'))] 
    texts = " ".join(splitwords)
    return texts

In [7]:
X_train_title = train_df.loc[:,'Title'].values
y_train_title = train_df.loc[:,['SentimentTitle']].values

X_train_headline = train_df.loc[:,'Headline'].values
y_train_headline = train_df.loc[:,['SentimentHeadline']].values

title_df=pd.DataFrame()
title_df['X_train_title']=X_train_title
title_df['y_train_title']=y_train_title

headline_df=pd.DataFrame()
headline_df['X_train_headline']=X_train_headline
headline_df['y_train_headline']=y_train_headline

title_df['X_train_title'] = title_df.X_train_title.apply(preprocess_text)
display(title_df.head())



,X_train_title,y_train_title
0,obama lays wreath arlington national cemetery,0.000000
1,look health chinese economy,0.208333
2,nouriel roubini: global economy back 2008,-0.425210
3,finland gdp expands q4,0.000000
4,"tourism, govt spending buoys thai economy january",0.000000


In [8]:
# !pip install -U spacy

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_curve


import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
lemma = WordNetLemmatizer()

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Bidirectional, LSTM, Dropout, BatchNormalization
from keras.layers.embeddings import Embedding

In [10]:
embeddings_index = dict()
f = open('V:/ML_projects/InterviewTask/NLP_pipeline/pretrain_embeddings/glove.6B.50d.txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [11]:
#Creating Embeddings for the titles

max_len_title = title_df.X_train_title.apply(lambda x: len(x.split())).max()

tok_title = Tokenizer()
tok_title.fit_on_texts(title_df.X_train_title)

vocab_size_title = len(tok_title.word_index) + 1

encoded_title = tok_title.texts_to_sequences(title_df.X_train_title)

padded_title = pad_sequences(encoded_title, maxlen=max_len_title, padding='post')

vocab_size_title = len(tok_title.word_index) + 1

title_embedding_matrix = np.zeros((vocab_size_title, 50))
for word, i in tok_title.word_index.items():
    t_embedding_vector = embeddings_index.get(word)
    if t_embedding_vector is not None:
        title_embedding_matrix[i] = t_embedding_vector

In [12]:
#Creating Embeddings for the Headlines

max_len_headline = headline_df.X_train_headline.apply(lambda x: len(x.split())).max()

tok_headline = Tokenizer()
tok_headline.fit_on_texts(headline_df.X_train_headline)
vocab_size_headline = len(tok_headline.word_index) + 1
encoded_headline = tok_headline.texts_to_sequences(headline_df.X_train_headline)
padded_headline = pad_sequences(encoded_headline, maxlen=max_len_headline, padding='post')

vocab_size_headline = len(tok_headline.word_index) + 1

headline_embedding_matrix = np.zeros((vocab_size_headline, 50))
for word, i in tok_headline.word_index.items():
    h_embedding_vector = embeddings_index.get(word)
    if h_embedding_vector is not None:
        headline_embedding_matrix[i] = h_embedding_vector

In [13]:
x_train_title, x_valid_title, Y_train_title, y_valid_title = train_test_split(padded_title, y_train_title, shuffle = True, test_size = 0.15)
x_train_headline, x_valid_headline, Y_train_headline, y_valid_headline = train_test_split(padded_headline, y_train_headline, shuffle = True, test_size = 0.15)


In [14]:
import math
from math import exp
import keras
# import tensorflow as tf

from keras import backend as K

def mod_tanh(x):
    return K.tanh(0.6*x)


# config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
# sess = tf.compat.v1.Session(config=config) 
# tf.compat.v1.keras.backend.set_session(sess)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [18]:
# Model for title
title_model = Sequential()
title_model.add(Embedding(vocab_size_title, 50, input_length=max_len_title, weights=[title_embedding_matrix], trainable=True))
title_model.add((LSTM(10, return_sequences=True)))
title_model.add(Dropout(0.3))
title_model.add(BatchNormalization())
title_model.add(LSTM(20, return_sequences=True))
title_model.add(Dropout(0.3))
title_model.add(BatchNormalization())
title_model.add(LSTM(20))
title_model.add(Dropout(0.3))
title_model.add(BatchNormalization())
title_model.add(Dense(64, activation='relu'))
title_model.add(Dense(8, activation='relu'))
title_model.add(Dense(1, activation=mod_tanh))
title_model.compile(loss='mse', optimizer='adam', metrics=['mse', 'mae'])
title_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 19, 50)            1406100   
                                                                 
 lstm_5 (LSTM)               (None, 19, 10)            2440      
                                                                 
 dropout_5 (Dropout)         (None, 19, 10)            0         
                                                                 
 batch_normalization_5 (Batc  (None, 19, 10)           40        
 hNormalization)                                                 
                                                                 
 dense_7 (Dense)             (None, 19, 8)             88        
                                                                 
 dense_8 (Dense)             (None, 19, 1)             9         
                                                      

In [19]:
# # Model for Headline
# headline_model = Sequential()
# headline_model.add(Embedding(vocab_size_headline, 100, input_length=max_len_headline, weights=[headline_embedding_matrix], trainable=True))
# headline_model.add(Bidirectional(LSTM(20, return_sequences=True)))
# headline_model.add(Dropout(0.3))
# headline_model.add(BatchNormalization())
# headline_model.add(Bidirectional(LSTM(20, return_sequences=True)))
# headline_model.add(Dropout(0.3))
# headline_model.add(BatchNormalization())
# headline_model.add(Bidirectional(LSTM(20)))
# headline_model.add(Dropout(0.3))
# headline_model.add(BatchNormalization())
# headline_model.add(Dense(64, activation='relu'))
# headline_model.add(Dense(64, activation='relu'))
# headline_model.add(Dense(1, activation=mod_tanh))
# headline_model.compile(loss='mse', optimizer='adam', metrics=['mse', 'mae'])

In [2]:
# title_model.fit(x_train_title, Y_train_title, epochs = 1,verbose=1)


In [3]:
# ######CNN
# headline_model = Sequential()
# headline_model.add(Embedding(vocab_size_headline, 100, input_length=max_len_headline, weights=[headline_embedding_matrix], trainable=True))
# title_model.add(Conv1D(filters=7, kernel_size=8, activation='relu'))
# title_model.add(MaxPooling1D(pool_size=2))
# title_model.add(Flatten())
# title_model.add(Dense(8, activation='relu'))
# title_model.add(Dense(1, activation=mod_tanh))
# print(title_model.summary())
# title_model.add(Dense(64, activation='relu'))
# title_model.add(Dense(1, activation=mod_tanh))
# title_model.compile(loss='mse', optimizer='adam', metrics=['mse', 'mae'])
# title_model.summary()